## Week 10 Homework Assignment MySQL

**Note: Question 4d is not clear.**

In [1]:
import pandas as pd
from config import MYSQL_USER, MYSQL_HOST, MYSQL_PASS
from sqlalchemy import create_engine

url = f'mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASS}@{MYSQL_HOST}:3306/sakila'
engine = create_engine(url)
df_schema = pd.read_sql('''
select 
    table_name, column_name 
from information_schema.columns 
where table_schema = "sakila"
''', engine)

### Experiment with sqlalchemy ORM

In [2]:
from sqlalchemy.ext.automap import automap_base
import redpanda
from sqlalchemy import func

Base = automap_base()
Base.prepare(engine, reflect=True)

Actor = Base.classes.actor
Address = Base.classes.address
Category = Base.classes.category
City = Base.classes.city
Country = Base.classes.country
Customer = Base.classes.customer
Film = Base.classes.film
FilmActor = Base.classes.film_actor
FilmText = Base.classes.film_text
Inventory = Base.classes.inventory
Language = Base.classes.language
Payment = Base.classes.payment
Rental = Base.classes.rental
Staff = Base.classes.staff
Store = Base.classes.store

Session = redpanda.orm.sessionmaker(bind=engine)
session = Session()

C:\Users\feng443\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\dialects\mysql\reflection.py:170: SAWarning: Did not recognize type 'geometry' of column 'location'
  (type_, name))


* 1a. Display the first and last names of all actors from the table `actor`. 

In [3]:
df_schema.query('table_name=="actor"')

,table_name,column_name
0,actor,actor_id
1,actor,first_name
2,actor,last_name
3,actor,last_update


In [4]:
pd.read_sql('desc actor', engine)

,Field,Type,Null,Key,Default,Extra
0,actor_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,first_name,varchar(45),NO,,None,
2,last_name,varchar(45),NO,MUL,None,
3,last_update,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


In [5]:
df_actor = pd.read_sql('''
select
    first_name,
    last_name
from actor
''', engine)
df_actor.head()

,first_name,last_name
0,PENELOPE,GUINESS
1,NICK,WAHLBERG
2,ED,CHASE
3,JENNIFER,DAVIS
4,JOHNNY,LOLLOBRIGIDA


In [6]:
session.query(Actor.first_name, Actor.last_name).frame().head()

,first_name,last_name
0,PENELOPE,GUINESS
1,NICK,WAHLBERG
2,ED,CHASE
3,JENNIFER,DAVIS
4,JOHNNY,LOLLOBRIGIDA


* 1b. Display the first and last name of each actor in a single column in upper case letters. Name the column `Actor Name`. 

In [7]:
df_actor_name = pd.read_sql('''
select 
    UPPER(concat(first_name, " ", last_name)) as "Actor Name" 
from actor
''', engine)
df_actor_name.head()

,Actor Name
0,PENELOPE GUINESS
1,NICK WAHLBERG
2,ED CHASE
3,JENNIFER DAVIS
4,JOHNNY LOLLOBRIGIDA


* 2a. You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe." What is one query would you use to obtain this information? 

In [8]:
df_joe = pd.read_sql('''
select 
    actor_id, 
    first_name, 
    last_name 
from actor 
where 
    first_name = "Joe"
''', engine)
df_joe

,actor_id,first_name,last_name
0,9,JOE,SWANK


In [9]:
session.query(Actor.actor_id, Actor.first_name, Actor.last_name).filter_by(first_name='Joe').frame()

,actor_id,first_name,last_name
0,9,JOE,SWANK


* 2b. Find all actors whose last name contain the letters `GEN`:

In [10]:
df_gen = pd.read_sql('''
select 
    actor_id, 
    first_name, 
    last_name 
from actor 
where 
    last_name like "%GEN%"
''', engine)
df_gen

,actor_id,first_name,last_name
0,14,VIVIEN,BERGEN
1,41,JODIE,DEGENERES
2,107,GINA,DEGENERES
3,166,NICK,DEGENERES


In [11]:
df_actor[df_actor.last_name.str.contains('GEN')]

,first_name,last_name
13,VIVIEN,BERGEN
40,JODIE,DEGENERES
106,GINA,DEGENERES
165,NICK,DEGENERES


In [12]:
session.query(
    Actor.actor_id, Actor.first_name, Actor.last_name
).filter(
    Actor.last_name.like('%GEN%')
).frame()

,actor_id,first_name,last_name
0,14,VIVIEN,BERGEN
1,41,JODIE,DEGENERES
2,107,GINA,DEGENERES
3,166,NICK,DEGENERES


* 2c. Find all actors whose last names contain the letters `LI`. This time, order the rows by last name and first name, in that order:

In [13]:
df_li_ordered = pd.read_sql('''
select 
    actor_id, first_name, last_name 
from actor 
where 
    last_name like "%LI%" 
order by 
    last_name, first_name
''', engine)
df_li_ordered

,actor_id,first_name,last_name
0,86,GREG,CHAPLIN
1,82,WOODY,JOLIE
2,34,AUDREY,OLIVIER
3,15,CUBA,OLIVIER
4,172,GROUCHO,WILLIAMS
5,137,MORGAN,WILLIAMS
6,72,SEAN,WILLIAMS
7,83,BEN,WILLIS
8,96,GENE,WILLIS
9,164,HUMPHREY,WILLIS


In [14]:
df_actor[df_actor.last_name.str.contains('LI')].sort_values(by=['last_name', 'first_name'])

,first_name,last_name
85,GREG,CHAPLIN
81,WOODY,JOLIE
33,AUDREY,OLIVIER
14,CUBA,OLIVIER
171,GROUCHO,WILLIAMS
136,MORGAN,WILLIAMS
71,SEAN,WILLIAMS
82,BEN,WILLIS
95,GENE,WILLIS
163,HUMPHREY,WILLIS


In [15]:
session.query(
    Actor.first_name, Actor.last_name
).filter(
    Actor.last_name.like('%LI%')
).order_by(
    Actor.last_name, Actor.first_name
).frame()

,first_name,last_name
0,GREG,CHAPLIN
1,WOODY,JOLIE
2,AUDREY,OLIVIER
3,CUBA,OLIVIER
4,GROUCHO,WILLIAMS
5,MORGAN,WILLIAMS
6,SEAN,WILLIAMS
7,BEN,WILLIS
8,GENE,WILLIS
9,HUMPHREY,WILLIS


* 2d. Using `IN`, display the `country_id` and `country` columns of the following countries: Afghanistan, Bangladesh, and China:

In [16]:
df_schema.query('table_name == "country"')

,table_name,column_name
24,country,country_id
25,country,country
26,country,last_update


In [17]:
pd.read_sql('desc country', engine)

,Field,Type,Null,Key,Default,Extra
0,country_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,country,varchar(50),NO,,None,
2,last_update,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


In [18]:
df_country_in3 = pd.read_sql('''
select
    country_id, country
from country
where
    country in ("Afghanistan", "Bangladesh", "China")
''', engine
)
df_country_in3

,country_id,country
0,1,Afghanistan
1,12,Bangladesh
2,23,China


In [19]:
session.query(
    Country.country_id, Country.country
).filter(
    Country.country.in_(["Afghanistan", "Bangladesh", "China"])
).frame()

,country_id,country
0,1,Afghanistan
1,12,Bangladesh
2,23,China


* 3a. Add a `middle_name` column to the table `actor`. Position it between `first_name` and `last_name`. Hint: you will need to specify the data type.

In [20]:
engine.execute('''
alter table actor
add column middle_name varchar(255) 
    after first_name
'''
)
pd.read_sql('desc actor', engine)

,Field,Type,Null,Key,Default,Extra
0,actor_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,first_name,varchar(45),NO,,None,
2,middle_name,varchar(255),YES,,None,
3,last_name,varchar(45),NO,MUL,None,
4,last_update,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


* 3b. You realize that some of these actors have tremendously long last names. Change the data type of the `middle_name` column to `blobs`.

In [21]:
engine.execute('''
alter table actor
change column middle_name middle_name blob
''')
pd.read_sql('desc actor', engine)

,Field,Type,Null,Key,Default,Extra
0,actor_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,first_name,varchar(45),NO,,None,
2,middle_name,blob,YES,,None,
3,last_name,varchar(45),NO,MUL,None,
4,last_update,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


* 3c. Now delete the `middle_name` column.

In [22]:
engine.execute('alter table actor drop column middle_name')
pd.read_sql('desc actor', engine)

,Field,Type,Null,Key,Default,Extra
0,actor_id,smallint(5) unsigned,NO,PRI,None,auto_increment
1,first_name,varchar(45),NO,,None,
2,last_name,varchar(45),NO,MUL,None,
3,last_update,timestamp,NO,,CURRENT_TIMESTAMP,on update CURRENT_TIMESTAMP


* 4a. List the last names of actors, as well as how many actors have that last name.  	

In [23]:
pd.read_sql('''
select 
    last_name, 
    count(*) actor_count 
from actor 
group by last_name 
order by 1
''', engine).head()

,last_name,actor_count
0,AKROYD,3
1,ALLEN,3
2,ASTAIRE,1
3,BACALL,1
4,BAILEY,2


In [24]:
pd.read_sql_table('actor', engine).groupby('last_name')['actor_id'].count().head()

last_name
AKROYD     3
ALLEN      3
ASTAIRE    1
BACALL     1
BAILEY     2
Name: actor_id, dtype: int64

In [25]:
session.query(
    Actor.last_name, 
    func.count(Actor.actor_id)
).group_by(
    Actor.last_name
).order_by(
    Actor.last_name
).frame().head()

,last_name,count_1
0,AKROYD,3
1,ALLEN,3
2,ASTAIRE,1
3,BACALL,1
4,BAILEY,2


* 4b. List last names of actors and the number of actors who have that last name, but only for names that are shared by at least two actors

In [26]:
pd.read_sql('''
select 
    last_name, 
    count(*) actor_count 
from actor 
group by last_name 
having actor_count > 1
order by 1
''', engine).head()

,last_name,actor_count
0,AKROYD,3
1,ALLEN,3
2,BAILEY,2
3,BENING,2
4,BERRY,3


In [27]:
pd.read_sql_table(
    'actor', engine
).groupby('last_name')['actor_id'].count().where(
    lambda x: x > 1
).dropna().head()

last_name
AKROYD    3.0
ALLEN     3.0
BAILEY    2.0
BENING    2.0
BERRY     3.0
Name: actor_id, dtype: float64

 Kind of interesting that Pandas convert the type to float64 here.

* 4c. Oh, no! The actor `HARPO WILLIAMS` was accidentally entered in the `actor` table as `GROUCHO WILLIAMS`, the name of Harpo's second cousin's husband's yoga teacher. Write a query to fix the record.

In [28]:
engine.execute('''
update actor 
set first_name = "HARPO"
where 
    first_name = "GROUCHO"
    and last_name = "WILLIAMS"
''')
pd.read_sql('''
select 
    * 
from actor 
where 
    last_name = "WILLIAMS"
''', engine)

,actor_id,first_name,last_name,last_update
0,72,SEAN,WILLIAMS,2006-02-15 04:34:33
1,137,MORGAN,WILLIAMS,2006-02-15 04:34:33
2,172,HARPO,WILLIAMS,2018-04-12 18:31:25


* 4d. Perhaps we were too hasty in changing `GROUCHO` to `HARPO`. It turns out that `GROUCHO` was the correct name after all! In a single query, if the first name of the actor is currently `HARPO`, change it to `GROUCHO`. Otherwise, change the first name to `MUCHO GROUCHO`, as that is exactly what the actor will be with the grievous error. BE CAREFUL NOT TO CHANGE THE FIRST NAME OF EVERY ACTOR TO `MUCHO GROUCHO`, HOWEVER! (Hint: update the record using a unique identifier.)

This question by itself, is confusing. From the Hint, an id is needed. However after the update the ID is not easy to get since name changes. If using first name = 'HARPO' and last_name = 'WILLIAMS' to identify the actor, the 'Otherwise, change the first name to MUCHO GROUCHO' part make not sense at all since the condition will never be true! 

In mysql, if all code running in one session here it can be done:


```SQL
select actor_id into @actor_id
from actor
where
    first_name = "GROUCHO"
    and last_name = "WILLIAMS";

update actor 
set first_name = "HARPO"
where actor_id = @actor_id;

update sakila.actor
set first_name = 
        case when first_name = "HARPO" then "GROUCHO"
        else "MUCHO GROUCHO" end
where actor_id = @actor_id;
```


In [29]:
pd.read_sql('''
select * from actor where first_name in ("HARPRO", "GROUCHO", "MUCHO GROUCHO")
''', engine)

,actor_id,first_name,last_name,last_update
0,78,GROUCHO,SINATRA,2006-02-15 04:34:33
1,106,GROUCHO,DUNST,2006-02-15 04:34:33


In [57]:
engine.execute('''
update sakila.actor
set first_name = 
        case when first_name = "HARPO" then "GROUCHO"
        else "MUCHO GROUCHO" end
where
/*
    first_name in ("HARPO", "MUCHO GROUCHO", "GROUCHO")
    and last_name = "WILLIAMS"
    */
    actor_id = 172
'''
)
pd.read_sql('''
select 
    * 
from actor 
where 
    last_name = "WILLIAMS"
''', engine)

,actor_id,first_name,last_name,last_update
0,72,SEAN,WILLIAMS,2006-02-15 04:34:33
1,137,MORGAN,WILLIAMS,2006-02-15 04:34:33
2,172,MUCHO GROUCHO,WILLIAMS,2018-04-12 18:42:30


* 5a. You cannot locate the schema of the `address` table. Which query would you use to re-create it? 
  * Hint: [https://dev.mysql.com/doc/refman/5.7/en/show-create-table.html](https://dev.mysql.com/doc/refman/5.7/en/show-create-table.html)

In [31]:
schema = pd.read_sql('''
select
    table_schema
from information_schema.tables
where
    table_name = "address"
''', engine)['table_schema'][0]
schema

'sakila'

In [32]:
print(pd.read_sql(f'''
show create table {schema}.address
''', engine)['Create Table'][0])

CREATE TABLE `address` (
  `address_id` smallint(5) unsigned NOT NULL AUTO_INCREMENT,
  `address` varchar(50) NOT NULL,
  `address2` varchar(50) DEFAULT NULL,
  `district` varchar(20) NOT NULL,
  `city_id` smallint(5) unsigned NOT NULL,
  `postal_code` varchar(10) DEFAULT NULL,
  `phone` varchar(20) NOT NULL,
  `location` geometry NOT NULL,
  `last_update` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (`address_id`),
  KEY `idx_fk_city_id` (`city_id`),
  SPATIAL KEY `idx_location` (`location`),
  CONSTRAINT `fk_address_city` FOREIGN KEY (`city_id`) REFERENCES `city` (`city_id`) ON UPDATE CASCADE
) ENGINE=InnoDB AUTO_INCREMENT=606 DEFAULT CHARSET=utf8


* 6a. Use `JOIN` to display the first and last names, as well as the address, of each staff member. Use the tables `staff` and `address`:

In [33]:
pd.read_sql('''
select
    s.first_name,
    s.last_name,
    a.address
from staff as s
left join address as a using(address_id)
order by
    last_name, first_name
''', engine).head()

,first_name,last_name,address
0,Mike,Hillyer,23 Workhaven Lane
1,Jon,Stephens,1411 Lillydale Drive


* 6b. Use `JOIN` to display the total amount rung up by each staff member in August of 2005. Use tables `staff` and `payment`. 
  	

In [34]:
pd.read_sql('''
select
    s.first_name,
    s.last_name,
    sum(p.amount) as total_amount
from staff as s
left join payment as p on
    p.staff_id = s.staff_id
    and payment_date >= '2005-08-01' 
    and payment_date < '2005-09-01' 
group by 1,2
order by 2,1
''', engine).head()

,first_name,last_name,total_amount
0,Mike,Hillyer,11853.65
1,Jon,Stephens,12218.48


* 6c. List each film and the number of actors who are listed for that film. Use tables `film_actor` and `film`. Use inner join.

Note: use left join in case some film show as as no-actor. Though this is unlikely to happen but it might reveal error in the data.

count(*), count(1) or count(fa.actor_id) makes no difference in this query. In some other query though, if there are duplicate, count(distinct fa.actor_id) might be better but could be dlower as well.

In [35]:
pd.read_sql('''
select
    f.title as film,
    count(1) as actor_count
from film as f
left join film_actor as fa using(film_id)
group by 1
order by 1
''', engine).head()

,film,actor_count
0,ACADEMY DINOSAUR,10
1,ACE GOLDFINGER,4
2,ADAPTATION HOLES,5
3,AFFAIR PREJUDICE,5
4,AFRICAN EGG,5


* 6d. How many copies of the film `Hunchback Impossible` exist in the inventory system?

MySQL does not enforce using group by if there is un-aggregated and aggregations but it is always good practice to do so. For example, this query still runs with out 'group by 1'.

In [36]:
sql = '''
select
    f.title as film,
    count(*) as copy_count
from film as f
left join inventory as i using(film_id)
where
    f.title = "Hunchback Impossible"
group by 1
'''
pd.read_sql(sql, engine).head()

,film,copy_count
0,HUNCHBACK IMPOSSIBLE,6


In [37]:
sql_on = '''
select
    f.title as film,
    count(*) as copy_count
from film as f
left join inventory as i on
    i.film_id = f.film_id
    and f.title = "Hunchback Impossible"
group by 1
'''
pd.read_sql(sql, engine).head()

,film,copy_count
0,HUNCHBACK IMPOSSIBLE,6


In [38]:
sql_sub = '''
select
    f.title as film,
    (
        select count(*)
        from inventory as i
        where
            i.film_id = f.film_id
    ) as copy_count
from film as f
where
    f.title = "Hunchback Impossible"
'''
pd.read_sql(sql_sub, engine).head()

,film,copy_count
0,HUNCHBACK IMPOSSIBLE,6


### Compare Execution Plans
The execution plan of case1 (condition on where) and case3 (where and subquery) is similar, while the execution plan for the condition in where cluase need to scan more rows.

In [39]:
pd.read_sql(f'explain {sql}', engine).head()

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,f,None,ref,idx_title,idx_title,767,const,1,100.0,Using index
1,1,SIMPLE,i,None,ref,idx_fk_film_id,idx_fk_film_id,2,sakila.f.film_id,4,100.0,Using index


In [40]:
pd.read_sql(f'explain {sql_on}', engine).head()

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,f,None,index,idx_title,idx_title,767,None,1000,100.0,Using index
1,1,SIMPLE,i,None,ref,idx_fk_film_id,idx_fk_film_id,2,sakila.f.film_id,4,100.0,Using where; Using index


Use subquery in select.

In [41]:
pd.read_sql(f'explain {sql_sub}', engine).head()

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,f,None,ref,idx_title,idx_title,767,const,1,100.0,Using index
1,2,DEPENDENT SUBQUERY,i,None,ref,idx_fk_film_id,idx_fk_film_id,2,sakila.f.film_id,4,100.0,Using index


### Compare Performances
The data set is small so the performance comparison might not be conclusive. However, putting the condition on on clause is actually slightly slower.

In [42]:
%timeit pd.read_sql(f'explain {sql}', engine).head()

3.33 ms ± 392 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%timeit pd.read_sql(f'explain {sql_on}', engine).head()

3.31 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%timeit pd.read_sql(f'explain {sql_sub}', engine).head()

3.48 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* 6e. Using the tables `payment` and `customer` and the `JOIN` command, list the total paid by each customer. List the customers alphabetically by last name:

In [45]:
pd.read_sql('''
select
    c.first_name, c.last_name,
    sum(amount) as total_amount
from customer as c
left join payment as p using(customer_id)
group by 1,2
order by 2, 1
''', engine).head()

,first_name,last_name,total_amount
0,RAFAEL,ABNEY,97.79
1,NATHANIEL,ADAM,133.72
2,KATHLEEN,ADAMS,92.73
3,DIANA,ALEXANDER,105.73
4,GORDON,ALLARD,160.68


* 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. As an unintended consequence, films starting with the letters `K` and `Q` have also soared in popularity. Use subqueries to display the titles of movies starting with the letters `K` and `Q` whose language is English. 

In [46]:
pd.read_sql('''
select
    f.title
from film as f
where
    f.title regexp "^[KQ]"
    and f.language_id = (
        select
            language_id
        from language
        where
            name = 'English'
    )
order by 1
''', engine)

,title
0,KANE EXORCIST
1,KARATE MOON
2,KENTUCKIAN GIANT
3,KICK SAVANNAH
4,KILL BROTHERHOOD
5,KILLER INNOCENT
6,KING EVOLUTION
7,KISS GLORY
8,KISSING DOLLS
9,KNOCK WARLOCK


* 7b. Use subqueries to display all actors who appear in the film `Alone Trip`.

In [47]:
pd.read_sql('''
select
    first_name, last_name
from actor
where
    actor_id in (
        select
            actor_id
        from film_actor
        where
            film_id = (
                select
                    film_id
                from film 
                where
                    title = 'Alone Trip'
            )
    )
order by 2,1
''', engine)

,first_name,last_name
0,RENEE,BALL
1,KARL,BERRY
2,LAURENCE,BULLOCK
3,ED,CHASE
4,CHRIS,DEPP
5,SPENCER,DEPP
6,WOODY,JOLIE
7,UMA,WOOD


* 7c. You want to run an email marketing campaign in Canada, for which you will need the names and email addresses of all Canadian customers. Use joins to retrieve this information.

For inner join putting condition in on cluase and where cause no performance difference but former has much better readability, especailly if using() clause can be utlized.

In [48]:
pd.read_sql('''
select
    customer.first_name,
    customer.last_name,
    customer.email
from customer
inner join address using(address_id)
inner join city    using(city_id)
inner join country using(country_id)
where
    country.country = 'Canada'
order by 2,1,3
''', engine)

,first_name,last_name,email
0,DERRICK,BOURQUE,DERRICK.BOURQUE@sakilacustomer.org
1,LORETTA,CARPENTER,LORETTA.CARPENTER@sakilacustomer.org
2,CURTIS,IRBY,CURTIS.IRBY@sakilacustomer.org
3,DARRELL,POWER,DARRELL.POWER@sakilacustomer.org
4,TROY,QUIGLEY,TROY.QUIGLEY@sakilacustomer.org


* 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. Identify all movies categorized as famiy films.

In [49]:
pd.read_sql('''
select
    film.title
from film
inner join film_category using(film_id)
inner join category      using(category_id)
where
    category.name = 'Family'
order by 1
''', engine).head()

,title
0,AFRICAN EGG
1,APACHE DIVINE
2,ATLANTIS CAUSE
3,BAKED CLEOPATRA
4,BANG KWAI


* 7e. Display the most frequently rented movies in descending order.

In [50]:
pd.read_sql('''
select
    film.title,
    count(rental.rental_id) rental_count
from film
inner join inventory using(film_id)
inner join rental    using(inventory_id)
group by film.title
order by rental_count desc
''', engine).head()

,title,rental_count
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,GRIT CLOCKWORK,32
3,FORWARD TEMPLE,32
4,SCALAWAG DUCK,32


* 7f. Write a query to display how much business, in dollars, each store brought in.

In [51]:
pd.read_sql('''
select
    store.store_id,
    sum(payment.amount) total_amount
from store
left join staff using(store_id)
left join payment using(staff_id)
group by store.store_id
order by store.store_id
''', engine)

,store_id,total_amount
0,1,33489.47
1,2,33927.04


* 7g. Write a query to display for each store its store ID, city, and country.

In [52]:
pd.read_sql('''
select
    store.store_id,
    city.city,
    country.country
from store
left join address using(address_id)
left join city using(city_id)
left join country using(country_id)
order by store.store_id
''', engine).head()

,store_id,city,country
0,1,Lethbridge,Canada
1,2,Woodridge,Australia


* 7h. List the top five genres in gross revenue in descending order. (**Hint**: you may need to use the following tables: category, film_category, inventory, payment, and rental.)  	

In [53]:
pd.read_sql('''
select
    category.name genre,
    sum(payment.amount) gross_revenue
from category
inner join film_category using(category_id)
inner join inventory using(film_id)
inner join rental using(inventory_id)
inner join payment using(rental_id)
group by genre
order by gross_revenue desc
limit 5
''', engine)

,genre,gross_revenue
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58


* 8a. In your new role as an executive, you would like to have an easy way of viewing the Top five genres by gross revenue. Use the solution from the problem above to create a view. If you haven't solved 7h, you can substitute another query to create a view.

In [54]:
view = 'top_five_genres'
engine.execute(f'drop view if exists {view}')
engine.execute(f'''
create view {view} as
select
    category.name genre,
    sum(payment.amount) gross_revenue
from category
inner join film_category using(category_id)
inner join inventory using(film_id)
inner join rental using(inventory_id)
inner join payment using(rental_id)
group by genre
order by gross_revenue desc
limit 5
''')
pd.read_sql_table(view, engine)

,genre,gross_revenue
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58


* 8b. How would you display the view that you created in 8a?

In [55]:
pd.read_sql(f'select * from {view}', engine)

,genre,gross_revenue
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58


* 8c. You find that you no longer need the view `top_five_genres`. Write a query to delete it.

In [56]:
engine.execute(f'drop view if exists {view}')